In [1]:
#set up credentials + imports
from office365.runtime.auth.user_credential import UserCredential
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.system_object_type import FileSystemObjectType
import os
from dotenv import load_dotenv

#set up credentials
load_dotenv(".env", override=True)
userCredential = UserCredential(os.getenv('user'),os.getenv('password') )

# Create a directory to store the text files
if not os.path.exists("pdf/"):
        os.mkdir("pdf/")

if not os.path.exists("pdf/hksinc.sharepoint.com/"):
        os.mkdir("pdf/hksinc.sharepoint.com/")

In [47]:
def downloadfile(file_url, ctx):
    
    download_path = os.path.join("pdf/hksinc.sharepoint.com/", os.path.basename(file_url))
    try:       
        file = ctx.web.get_file_by_server_relative_url(file_url)
        with open(download_path, "wb") as local_file:
            file = ctx.web.get_file_by_server_relative_url(file_url).download(local_file).execute_query()
        print("[Ok] file has been downloaded into: {0}".format(download_path))
    except:
        print("file not found")

In [59]:
import json

def enum_folder(parent_folder, fn, ctx):
    """
    :type parent_folder: Folder
    :type fn: (File)-> None
    """
    parent_folder.expand(["Files", "Folders"]).get().execute_query()
    print(len(parent_folder.files))
    for file in parent_folder.files:  # type: File
        
        #download if it is a pdf
        url = file.properties['ServerRelativeUrl'];
        if 'pdf' in url:
            fn(file , ctx)
            print_file(file, ctx)
            #downloadfile(url, ctx)
    for folder in parent_folder.folders:  # type: Folder
        enum_folder(folder, fn, ctx)


def print_file(f,ctx):
    """
    :type f: File
    """
    print(f.properties['ServerRelativeUrl'])

def print_props(f, ctx):
    json_object = json.dumps(f.properties, indent = 4) 
    print(json_object)
    with open("pdf/sample.json", "a") as outfile:
        json.dump(f.properties, outfile)
        outfile.write('\n')
    # for p in f.properties:
    #     print(p, f.properties[p])


In [46]:
def findDocs(target_folder_url, ctx):
    root_folder = ctx.web.get_folder_by_server_relative_path(target_folder_url)
    enum_folder(root_folder, print_file, ctx)

In [60]:
import pandas as pd
#read from csv, clean and filter
sitesdf= pd.read_csv("knowledgesites.csv",keep_default_na=False)

#iterate sites        
for index, row in sitesdf.head(3).iterrows():
    site_url = row["url"]
    
    ctx = ClientContext(site_url.strip()).with_credentials(userCredential)
    web = ctx.web
    ctx.load(web)
    try :
        ctx.execute_query()
    except :
         continue
    findDocs("Shared Documents", ctx)
    

2183
https://hksinc.sharepoint.com/sites/research/sites/research/Shared Documents/Sustainable Building Design and impacts on Mental Health.pdf
https://hksinc.sharepoint.com/sites/research/sites/research/Shared Documents/Sustainable Building Design and impacts on Mental Health.pdf
https://hksinc.sharepoint.com/sites/research/sites/research/Shared Documents/Gensler_Inclusion-Utopia_35521.pdf
https://hksinc.sharepoint.com/sites/research/sites/research/Shared Documents/Gensler_Inclusion-Utopia_35521.pdf
https://hksinc.sharepoint.com/sites/research/sites/research/Shared Documents/2017_HKS_Design Diagnostic of a Cancer Center.pdf
https://hksinc.sharepoint.com/sites/research/sites/research/Shared Documents/2017_HKS_Design Diagnostic of a Cancer Center.pdf
https://hksinc.sharepoint.com/sites/research/sites/research/Shared Documents/Guiding Principles Research_large.pdf
https://hksinc.sharepoint.com/sites/research/sites/research/Shared Documents/Guiding Principles Research_large.pdf
https://hks

In [67]:
from office365.sharepoint.taxonomy.field import TaxonomyField

def list_taxonomy_fields(itemlist):
    fields = itemlist.fields.get().execute_query()
    #get all fields
    taxonomy_fields = []
    for f in fields:  # type: Field
        #find only taxonomy types
        # print(f)
        # print(f"Field internal name {f.internal_name}")
        # print(f"Field title {f.title}")
        if type(f) is TaxonomyField:
            # print(f)
            # print(f"Field internal name {f.internal_name}")
            # print(f"Field title {f.title}")
            taxonomy_fields.append(f)
    return taxonomy_fields

In [65]:
def getalllists(ctx):
    lists = ctx.web.lists.get_all().execute_query()
    print(lists)
    
    for l in lists:
        # c = l.content_types.get_all().execute_query()
        # for t in c:
        #     print(t.description)
        doclib = "no"
        if l.properties["BaseType"] == 1:
            doclib = "yes"
            #print("List title: {0}. # items: {1}".format(l.title, l.properties["ItemCount"]))
    return lists
        

In [34]:
import json
def query_items_byfield(target_list, tax_field):
    """
    :type target_list: office365.sharepoint.lists.list.List
    """
    #https://hksinc.sharepoint.com/sites/research/Shared%20Documents/ + LinkFilename is url
    items = target_list.items.get().select(["LinkFilename", tax_field.internal_name]).execute_query() #tax_field.internal_name, 
    for t in items:
        json_object = json.dumps(t.properties) 
        print(json_object)


In [70]:

from office365.sharepoint.taxonomy.field import TaxonomyField

site_url = "https://hksinc.sharepoint.com/sites/InclusiveDesign"
ctx = ClientContext(site_url).with_credentials(userCredential)
lists = getalllists(ctx)
for l in lists:
    itemlist = ctx.web.lists.get_by_title(l.title) #Research Contribution Library
    tax_fields = list_taxonomy_fields(itemlist)  
    if len(tax_fields) > 0: 
        print(l.title, len(tax_fields) )
      
#itemlist = ctx.web.lists.get_by_title("Research Knowledge Library") #Research Contribution Library
#tax_fields = list_taxonomy_fields(itemlist)
#query for a specific taxonomy
#query_items_byfield(itemlist, tax_fields[0])
# too many columns to get all items need to work on field types
# items = tasks_list.items.get().execute_query()

[<office365.sharepoint.lists.list.List object at 0x000001F52C164F40>, <office365.sharepoint.lists.list.List object at 0x000001F52C166D10>, <office365.sharepoint.lists.list.List object at 0x000001F52C165690>, <office365.sharepoint.lists.list.List object at 0x000001F52C165210>, <office365.sharepoint.lists.list.List object at 0x000001F52C1667D0>, <office365.sharepoint.lists.list.List object at 0x000001F52C164A60>, <office365.sharepoint.lists.list.List object at 0x000001F52C1649D0>, <office365.sharepoint.lists.list.List object at 0x000001F513329120>, <office365.sharepoint.lists.list.List object at 0x000001F512F54BE0>, <office365.sharepoint.lists.list.List object at 0x000001F52C164C40>, <office365.sharepoint.lists.list.List object at 0x000001F52A6F38B0>, <office365.sharepoint.lists.list.List object at 0x000001F52C1647C0>, <office365.sharepoint.lists.list.List object at 0x000001F52C164BE0>, <office365.sharepoint.lists.list.List object at 0x000001F52A6F3850>, <office365.sharepoint.lists.list.

In [25]:
#checking the process on the base url and relative url
site_url = "https://hksinc.sharepoint.com/sites/InclusiveDesign"
ctx = ClientContext(site_url).with_credentials(userCredential)
web = ctx.web
ctx.load(web)
print(ctx.base_url)
ctx.execute_query()
downloadfile("/sites/research/Shared Documents/One decor tendency you must avoid in order to gain a 100% happy home.pdf", ctx)

https://hksinc.sharepoint.com/sites/InclusiveDesign


NameError: name 'downloadfile' is not defined

In [ ]:
# importing required modules
from PyPDF2 import PdfReader

def read_pdf(file):
    # creating a pdf reader object
    reader = PdfReader('pdf/hksinc.sharepoint.com/05_Interviews_Documentation Example.pdf')
    
    # printing number of pages in pdf file
    print(len(reader.pages))
    
    # getting a specific page from the pdf file
    page = reader.pages[0]
    
    # extracting text from page
    text = page.extract_text()
    print(text)